In [158]:
from __future__ import division, absolute_import, print_function
%load_ext autoreload
%autoreload 2

import os 
#os.chdir(r"D:\data\ortoCC\design-with-alignments\mason_small")
print(os.getcwd()) 
    
import hepran
import hepran.bzipscore as bz
import hepran.bcipa as bc
import hepran.utils as u
import hepran.registers as r

import hepran.agadir as ag

import pandas as pd
import numpy as np
import seaborn as sns
    
%matplotlib inline

import matplotlib as mpl
import matplotlib.pylab as plt

from  score_utils import *
    
CCO = os.environ.get("CCO",r"C:\Projekti_KI\ortoCC\CoiledCoilOrtho")

set_dir = "D:\data\ortoCC\design-with-alignments\4heptade-7\!OUT_bcf"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
d:\data\ortoCC\fine_tune_scoring_function


In [159]:
from interactive_set_plot import *

import bokeh
import bokeh.resources
import bokeh.plotting as bp
from bokeh.models import HoverTool
from IPython.core.display import display, HTML
import hepran
bp.output_notebook()

import sklearn
from sklearn import linear_model
from sklearn.externals import joblib

from IPython.display import display, HTML

Loading BokehJS ...

In [160]:
df_dna=pd.read_excel('data/all.xlsx', index_col=[0,1,4,5])

In [161]:
O_DNA = df_dna["mean_RD 	sd_RD 	med_RD 	sum_RD 	bcnum 	ln_med_RD 	ln_mean_RD on_target score_bc7".split()].query("(condition == 6) and (bcf_set=='bA')")
O_DNA = O_DNA.reset_index().set_index("ID1 ID2".split())

In [162]:
O_DNA['IDs']=O_DNA.index
O_DNA = O_DNA.assign(**O_DNA.index.to_frame())
O_DNA['RD_Tm'] = ln_mean_RD_to_Tm(O_DNA.ln_mean_RD)
O_DNA['score'] = O_DNA['score_bc7']
O_DNA['seq1'] = ''
O_DNA['seq2'] = ''


In [163]:
seqs = u.load_fasta('data/all-8k.fasta')

In [164]:
def join_seqs(row, seqs):
    #print(row)
    ID1 = row['ID1']
    ID2 = row['ID2']
    row['seq1'] = seqs[ID1]
    row['seq2'] = seqs[ID2]
    row = get_formated_seq(row)
    return row

In [165]:
O_DNA = O_DNA.apply(join_seqs, axis=1, args=(seqs,))

In [166]:
tooltips = [
    ('ID1, ID2', '@IDs'),
    ('RD_Tm', '@RD_Tm'),
    ('score', '@score'),
    ('ln_mean_RD', '@ln_mean_RD'),
    ('bcnum', '@bcnum'),
    ('sd_RD', '@sd_RD'),
    ('seq1', '@seq1_disp{safe}'),
    ('seq2', '@seq2_disp{safe}'),
]

p = draw_scatter_interactive('RD_Tm','score_bc7',O_DNA, y_range=(0,80),                                
                                 title='RD_Tm vs bcipa', save_to_file=False, tooltips=tooltips)


bp.show(p)


In [168]:
O_DNA['avg_HP1']=O_DNA.seq1.apply(bc.get_HP)
O_DNA['avg_HP2']=O_DNA.seq2.apply(bc.get_HP)
O_DNA['avg_HP_12']=(O_DNA.avg_HP1+O_DNA.avg_HP2)/2

O_DNA['HP1']=O_DNA.seq1.apply(bc.get_HP, average=False)
O_DNA['HP2']=O_DNA.seq2.apply(bc.get_HP, average=False)
O_DNA['HP_12']=(O_DNA.HP1+O_DNA.HP2)

In [169]:
#O_DNA['sd_RD'.split()].plot.hist(alpha=0.5)
O_DNA['sd_RD_norm'] = O_DNA['sd_RD']/(O_DNA['sd_RD'].mean())

In [170]:
#O_DNA['avg_HP1 avg_HP2'.split()].plot.hist(alpha=0.5)

In [171]:
O_DNA.to_csv('data/DNA_round0_fiting.csv')

In [172]:
%%time
df = pd.read_csv('data/DNA_round0_fiting.csv', index_col=[0,1])
O_DNA = O_DNA.apply(get_CC_features, axis=1)
O_DNA.to_csv('data/DNA_round0_fiting.csv')